In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1577128373658_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.version

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2.4.4'

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1577128373658_0001,pyspark,idle,Link,Link,✔


In [4]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from pyspark.sql.functions import col, input_file_name, udf
from pyspark.sql.functions import sum as spark_sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

def run_sql_pandas(statement):
    return run_sql(statement).toPandas()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sometimes using a wildcard for the files does not work.  In such cases, you can either manually list all the files as an array, or you can use
   
   files = "s3://topic-sentiment-1/combined/"
   
However, the latter method renders the source_file column useless, and so most of the analysis won't work.

In [9]:
files = "s3://topic-sentiment-1/combined/*.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
raw_df = spark.read.json(files)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
raw_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- language: string (nullable = true)
 |-- localpath: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- title_rss: string (nullable = true)
 |-- url: string (nullable = true)

In [12]:
raw_df = raw_df.withColumn("source_file", input_file_name())

def file_only(path):
    return path.split('/')[-1]

file_only_udf = udf(file_only)
raw_df = raw_df.withColumn("source_file", file_only_udf(col("source_file")))

def user_agent(path):
    return "googlebot" if "googlebot" in path else "default"

user_agent_udf = udf(user_agent)
raw_df = raw_df.withColumn("user_agent", user_agent_udf(col("source_file")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
run_sql('drop database if exists topic cascade')
run_sql('create database topic')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [14]:
permanent_table_name = 'topic.articles'
raw_df.write.format("parquet").saveAsTable(permanent_table_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
by_file = run_sql_pandas('''
    SELECT user_agent, source_domain, count(*) as total_count
    FROM topic.articles
    GROUP by source_domain, user_agent
''')
by_file.sort_values(by=['source_domain', 'user_agent']).head(300)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    user_agent           source_domain  total_count
12   default    ajc.com                 2769       
66   googlebot  ajc.com                 911        
6    default    americanthinker.com     43382      
17   googlebot  americanthinker.com     35326      
97   default    apnews.com              27032      
5    googlebot  apnews.com              84715      
2    default    axios.com               32906      
71   googlebot  axios.com               57993      
99   default    bbc.com                 25254      
87   googlebot  bbc.com                 73419      
93   googlebot  bloomberg.com           4          
78   default    boston.com              22903      
0    googlebot  boston.com              41866      
8    default    bostonglobe.com         88         
31   googlebot  bostonglobe.com         91         
68   default    breitbart.com           17986      
59   googlebot  breitbart.com           29733      
52   default    cbsnews.com             8582       
33   googleb

In [16]:
duplicates = run_sql_pandas('''
    SELECT source_domain, count(*) as total_count FROM (
    SELECT url, source_domain, count(*) FROM topic.articles
    GROUP BY url, source_domain
    HAVING count(*) > 1) a GROUP BY source_domain
''').sort_values(by='source_domain')
duplicates.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             source_domain  total_count
20  ajc.com                 104        
9   americanthinker.com     9239       
45  apnews.com              23756      
35  axios.com               33675      
22  bbc.com                 25240      
17  boston.com              7769       
42  breitbart.com           17689      
29  cbsnews.com             7858       
3   chicago.suntimes.com    14387      
34  chicagotribune.com      15521      
43  chron.com               11472      
46  cnbc.com                1778       
23  dallasnews.com          29761      
32  denverpost.com          12622      
19  economist.com           66         
44  fivethirtyeight.com     13709      
27  forbes.com              580        
15  foreignpolicy.com       29494      
25  foxnews.com             31098      
0   ft.com                  9020       
18  latimes.com             4311       
6   msnbc.com               875        
2   nationalreview.com      62         
12  nbcnews.com             26284      


In [18]:
# this is a trick: I want to display the count of duplicates along side the counts for each user agent, so I fake these
duplicates['user_agent'] = 'duplicates'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
all_files = by_file.append(duplicates)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/tmp/1577128580859-0/local/lib64/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,

In [20]:
all_sites = all_files.pivot(index='source_domain', columns='user_agent').fillna(0)
all_sites

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                       total_count                     
user_agent                 default duplicates googlebot
source_domain                                          
ajc.com                 2769.0      104.0      911.0   
americanthinker.com     43382.0     9239.0     35326.0 
apnews.com              27032.0     23756.0    84715.0 
axios.com               32906.0     33675.0    57993.0 
bbc.com                 25254.0     25240.0    73419.0 
bloomberg.com           0.0         0.0        4.0     
boston.com              22903.0     7769.0     41866.0 
bostonglobe.com         88.0        0.0        91.0    
breitbart.com           17986.0     17689.0    29733.0 
cbsnews.com             8582.0      7858.0     24555.0 
chicago.suntimes.com    33446.0     14387.0    44.0    
chicagotribune.com      15850.0     15521.0    47503.0 
chron.com               12294.0     11472.0    37487.0 
cnbc.com                1873.0      1778.0     15291.0 
dailykos.com            0.0         0.0        2

We have completed crawls for many sites with both Googlebot as the user agent, and with the default user agent.  Although the date periods do not completely overlap, and the depth of the crawl varies, we can draw some conclusions.  In general, googlebot was allowed to run longer than anything else, so those counts could be expected to be higher in general

Note that user agent for washingtonpost.com is not correct.  I only got results with the default user agent.  A quirk of the data collection mislabeled these as using Googlebot.


In [21]:
def choose_user_agent(row):
    row = row['total_count']
    no_default = (row.default < 800)
    no_google = (row.googlebot < 800)
    if no_default and no_google:
        return "not enough data"
    if no_default and (not no_google):
        return "googlebot"
    if no_google and (not no_default):
        return "default"
    dup_ratio = row.duplicates / (row.googlebot + row.default)
    if dup_ratio < 0.1:
        return "both"
    default_ratio = row.default / row.googlebot
    if default_ratio > 1:
        return "default"
    else:
        return "googlebot"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
all_sites['choice'] = all_sites.apply(choose_user_agent, axis=1)
all_sites.head(300)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                       total_count                                choice
user_agent                 default duplicates googlebot                 
source_domain                                                           
ajc.com                 2769.0      104.0      911.0     both           
americanthinker.com     43382.0     9239.0     35326.0   default        
apnews.com              27032.0     23756.0    84715.0   googlebot      
axios.com               32906.0     33675.0    57993.0   googlebot      
bbc.com                 25254.0     25240.0    73419.0   googlebot      
bloomberg.com           0.0         0.0        4.0       not enough data
boston.com              22903.0     7769.0     41866.0   googlebot      
bostonglobe.com         88.0        0.0        91.0      not enough data
breitbart.com           17986.0     17689.0    29733.0   googlebot      
cbsnews.com             8582.0      7858.0     24555.0   googlebot      
chicago.suntimes.com    33446.0     14387.0    44.0